###Checkpoints
* Modelling - Done
* yaml file - Done
* Multi GPU training - Done
* CPU inferencing which outputs F1 score - Done
* Logging of training - Done
* Documentation of project - PPT Done Report Remaining

In [6]:
!pip install transformers --quiet
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn
from sklearn import metrics
import transformers
from transformers import AutoTokenizer
from sklearn.preprocessing import LabelEncoder
from transformers import TFRobertaModel
import json
import matplotlib.pyplot as plt
import random
import seaborn as sns
from sklearn.metrics import f1_score
import warnings

warnings.filterwarnings('ignore')

In [2]:
# Detect hardware, return appropriate distribution strategy, if tpus are available tpus are used
# Else if gpus are available gpus are used. If neither are available computation is done with CPUs
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    # Distribution strategy if tpus are available and is to be used
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print('Using {} TPUs'.format(strategy.num_replicas_in_sync))

elif tf.config.list_physical_devices('GPU'):
    # Distribution strategy in case of multiple GPUs
    strategy = tf.distribute.MirroredStrategy()
    print('Using {} GPUs'.format(strategy.num_replicas_in_sync))

else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.MirroredStrategy()
    print('No GPU nor TPU. Running on CPU')

AUTOTUNE = tf.data.experimental.AUTOTUNE

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


Running on TPU  grpc://10.10.150.202:8470
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.10.150.202:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.10.150.202:8470


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Using 8 TPUs


In [3]:
BATCH_SIZE = 512
MAX_LEN = 13
NUM_ACTION = 6
NUM_OBJECT = 14
NUM_LOCATION = 4

# TRAINING (IGNORE IF NOT NEEDED)

In [113]:
train_df = pd.read_csv('/content/drive/MyDrive/Backup_file/train_data.csv')
val_df = pd.read_csv('/content/drive/MyDrive/Backup_file/valid_data.csv')

In [114]:
action_enc = LabelEncoder()
action_train = action_enc.fit_transform(train_df['action'])
action_val = action_enc.transform(val_df['action'])

object_enc = LabelEncoder()
object_train = object_enc.fit_transform(train_df['object'])
object_val = object_enc.transform(val_df['object'])

location_enc = LabelEncoder()
location_train = location_enc.fit_transform(train_df['location'])
location_val = location_enc.transform(val_df['location'])

texts = train_df['transcription'].values
texts = list(texts)
val_texts = val_df['transcription'].values
val_texts = list(val_texts)

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
train_data = tokenizer(texts, max_length=MAX_LEN, padding='max_length', truncation=True, return_tensors='tf')
val_data = tokenizer(val_texts, max_length=MAX_LEN, padding='max_length', truncation=True, return_tensors='tf')

In [116]:
y_train = {'action': action_train, 'object':object_train, 'location': location_train}
y_val = {'action': action_val, 'object':object_val, 'location': location_val}

train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_data), y_train)).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_data), y_val)).batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)

In [117]:
%%time
def build_model():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32, name='input_ids')
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32, name='attention_mask')
    bert_model = TFRobertaModel.from_pretrained("roberta-base")
    x = bert_model(ids,attention_mask=att)
    x1 = tf.keras.layers.Flatten()(x[1])
    x1 = tf.keras.layers.Dense(NUM_ACTION, name='action')(x1)

    x2 = tf.keras.layers.Flatten()(x[1])
    x2 = tf.keras.layers.Dense(NUM_OBJECT, name='object')(x2)

    x3 = tf.keras.layers.Flatten()(x[1])
    x3 = tf.keras.layers.Dense(NUM_LOCATION, name='location')(x3)
    model = tf.keras.models.Model(inputs=[ids, att], outputs=[x1,x2,x3])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5, clipnorm=1.),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.metrics.SparseCategoricalAccuracy(),tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name='Top_3')],
        )
    return model

with strategy.scope():
    model = build_model()

history=model.fit(train_dataset, validation_data=val_dataset, epochs=6, verbose=1)

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Epoch 1/6


INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 13) dtype=int32>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 13) dtype=int32>, <tf.Tensor 'cond/Identity_16:0' shape=(None,) dtype=int64>, <tf.Tensor 'cond/Identity_24:0' shape=(None,) dtype=int64>, <tf.Tensor 'cond/Identity_32:0' shape=(None,) dtype=int64>]
INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 13) dtype=int32>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 13) dtype=int32>, <tf.Tensor 'cond/Identity_16:0' shape=(None,) dtype=int64>, <tf.Tensor 'cond/Identity_24:0' shape=(None,) dtype=int64>, <tf.Tensor 'cond/Identity_32:0' shape=(None,) dtype=int64>]


23/23 [==============================] - ETA: 0s - loss: 4.8165 - action_loss: 1.6378 - object_loss: 2.1320 - location_loss: 1.0466 - action_sparse_categorical_accuracy: 0.3177 - action_Top_3: 0.6705 - object_sparse_categorical_accuracy: 0.3444 - object_Top_3: 0.5988 - location_sparse_categorical_accuracy: 0.5713 - location_Top_3: 0.9159

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 13) dtype=int32>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 13) dtype=int32>, <tf.Tensor 'cond/Identity_16:0' shape=(None,) dtype=int64>, <tf.Tensor 'cond/Identity_24:0' shape=(None,) dtype=int64>, <tf.Tensor 'cond/Identity_32:0' shape=(None,) dtype=int64>]


23/23 [==============================] - 182s 4s/step - loss: 4.8165 - action_loss: 1.6378 - object_loss: 2.1320 - location_loss: 1.0466 - action_sparse_categorical_accuracy: 0.3177 - action_Top_3: 0.6705 - object_sparse_categorical_accuracy: 0.3444 - object_Top_3: 0.5988 - location_sparse_categorical_accuracy: 0.5713 - location_Top_3: 0.9159 - val_loss: 2.7310 - val_action_loss: 1.0437 - val_object_loss: 1.0587 - val_location_loss: 0.6286 - val_action_sparse_categorical_accuracy: 0.6373 - val_action_Top_3: 0.9589 - val_object_sparse_categorical_accuracy: 0.7412 - val_object_Top_3: 0.8066 - val_location_sparse_categorical_accuracy: 0.7880 - val_location_Top_3: 0.9962
Epoch 2/6
23/23 [==============================] - 4s 198ms/step - loss: 1.8906 - action_loss: 0.7065 - object_loss: 0.7782 - location_loss: 0.4058 - action_sparse_categorical_accuracy: 0.8308 - action_Top_3: 0.9763 - object_sparse_categorical_accuracy: 0.8108 - object_Top_3: 0.8975 - location_sparse_categorical_accuracy: 

In [123]:
SAVE_PATH = '/content/drive/MyDrive/Saarthi/'
model.save_weights(SAVE_PATH+'model.h5')
np.save(SAVE_PATH+'action_encoder.npy', action_enc.classes_)
np.save(SAVE_PATH+'object_encoder.npy', object_enc.classes_)
np.save(SAVE_PATH+'location_encoder.npy', location_enc.classes_)

# PREDICTION

In [11]:
#path to test
PATH = '/content/drive/MyDrive/Saarthi/valid_data.csv'
#folder containing model and the encoders
SAVE_PATH = '/content/drive/MyDrive/Saarthi/'

test_df = pd.read_csv(PATH)

action_enc = LabelEncoder()
action_enc.classes_ = np.load(SAVE_PATH+'action_encoder.npy', allow_pickle=True)
action_test = action_enc.transform(test_df['action'])

object_enc = LabelEncoder()
object_enc.classes_ = np.load(SAVE_PATH+'object_encoder.npy', allow_pickle=True)
object_test = object_enc.transform(test_df['object'])

location_enc = LabelEncoder()
location_enc.classes_ = np.load(SAVE_PATH+'location_encoder.npy', allow_pickle=True)
location_test = location_enc.transform(test_df['location'])

test_texts = test_df['transcription'].values
test_texts = list(test_texts)
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

MAX_LEN = 13
test_data = tokenizer(test_texts, max_length=MAX_LEN, padding='max_length', truncation=True, return_tensors='tf')
y_test = {'action': action_test, 'object':object_test, 'location': location_test}
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_data), y_test)).batch(BATCH_SIZE)

def build_model():
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32, name='input_ids')
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32, name='attention_mask')
    bert_model = TFRobertaModel.from_pretrained("roberta-base")
    x = bert_model(ids,attention_mask=att)
    x1 = tf.keras.layers.Flatten()(x[1])
    x1 = tf.keras.layers.Dense(NUM_ACTION, name='action')(x1)

    x2 = tf.keras.layers.Flatten()(x[1])
    x2 = tf.keras.layers.Dense(NUM_OBJECT, name='object')(x2)

    x3 = tf.keras.layers.Flatten()(x[1])
    x3 = tf.keras.layers.Dense(NUM_LOCATION, name='location')(x3)
    model = tf.keras.models.Model(inputs=[ids, att], outputs=[x1,x2,x3])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5, clipnorm=1.),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.metrics.SparseCategoricalAccuracy(),tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name='Top_3')],
        )
    return model

with strategy.scope():
    model = build_model()

model.load_weights(SAVE_PATH+'model.h5')
ans = model.predict(test_dataset)

action_preds = action_enc.inverse_transform(np.argmax(ans[0],axis=1))
object_preds = object_enc.inverse_transform(np.argmax(ans[1],axis=1))
location_preds = location_enc.inverse_transform(np.argmax(ans[2],axis=1))

pred_df = pd.DataFrame({'input':test_df['transcription'],'action':test_df['action'],'object':test_df['object'],'location':test_df['location'],
                        'action_preds':action_preds, 'object_preds':object_preds, 'location_preds':location_preds})

# micro f1 score
action_f1 = f1_score(pred_df['action_preds'],pred_df['action'], average='micro')
object_f1 = f1_score(pred_df['object_preds'],pred_df['object'], average='micro')
location_f1 = f1_score(pred_df['location_preds'],pred_df['location'], average='micro')
print('F1 score for action-->',action_f1)
print('F1 score for object-->',object_f1)
print('F1 score for location-->',location_f1)
pred_df

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 13) dtype=int32>, <tf.Tensor 'cond/Ident

F1 score for action--> 1.0
F1 score for object--> 1.0
F1 score for location--> 1.0


,input,action,object,location,action_preds,object_preds,location_preds
0,Turn on the lights,activate,lights,none,activate,lights,none
1,Turn off the lights,deactivate,lights,none,deactivate,lights,none
2,Change language,change language,none,none,change language,none,none
3,Pause the music,deactivate,music,none,deactivate,music,none
4,Resume,activate,music,none,activate,music,none
...,...,...,...,...,...,...,...
3113,Lights on,activate,lights,none,activate,lights,none
3114,Switch off the lights,deactivate,lights,none,deactivate,lights,none
3115,Turn the lights off,deactivate,lights,none,deactivate,lights,none
3116,Lights off,deactivate,lights,none,deactivate,lights,none
